In [55]:
import pandas as pd
import numpy as np
import math


# Packages for webscraping
import os, random, sys, time
from urllib.parse import urlparse
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

#Packages for Reading PDFs
import camelot

import os

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

In [12]:
driver = webdriver.Chrome('chromedriver.exe')

In [13]:
base_url = 'https://missionantyodaya.nic.in/preloginScoreOfGp2020.html'

In [ ]:
# LOOPING ACROSS 34 DISTRICTS IN TELANGANA. IF YOU WANT ALL INDIA, GENERATE A DYNAMIC COUNTER TO COUNT NUMBER OF DISTRICTS FOR EACH STATE
for i in range(1,34,1):
    #OPEN URL
    driver.get(base_url)
    driver.implicitly_wait(10)
    # SELECT STATE -- DEFAULTING TO TELANGANA -- at No.33
    Select(driver.find_element_by_xpath("//select[@name='stateCode']")).options[33].click()
    print('District No:',i)
    time.sleep(2)
    # SELECT EACH DISTRICT and HIT SUBMIT
    Select(driver.find_element_by_xpath("//select[@name='districtCode']")).options[i].click()
    driver.find_element_by_xpath("//button[@id='SubmitButton']").click()
    time.sleep(5)


    try:
        no_villages = int(driver.find_element_by_xpath("//div[@id='example_info']").text.split('entries')[0].strip()[-3:])
        print('Number of Villages in District: ',no_villages )
        max_pages = int(np.ceil(no_villages/10))
    except:
        continue
    
    #DOWNLOAD THE MASTER EXCEL FOR EACH DISTRICT
    driver.find_element_by_xpath("//button[@class='dt-button buttons-excel buttons-html5 btn btn-default btn-xs']").click()
    
    # HIT THE DOWNLOAD BUTTON FOR EACH VILLAGE
    Download_options = driver.find_elements_by_xpath("//*[contains(text(), 'Download')]")
    no_downloads = len(Download_options)
    for k in range(no_downloads):
        Download_options[k].click()
    
    
    # CRAWL ACROSS EACH PAGE and REPEAT ABOVE STEP
    for w in range(2,max_pages+1):
        print('Page No:',w)
        driver.find_element_by_xpath("//a[@class='paginate_button next']").click()
        Download_options = driver.find_elements_by_xpath("//*[contains(text(), 'Download')]")
        no_downloads = len(Download_options)
        for k in range(no_downloads):
            Download_options[k].click()
        time.sleep(5)

- Make sure to change your downloads folder. 

# READING DOWNLOADED PDFs

In [68]:
files = [f for f in os.listdir('Antyodaya_Telangana') if f[-3:]=='pdf']
files[:5]

['00194909_f_20201209162305.pdf',
 '00194911_f_20201223125603.pdf',
 '00194913_f_20201223125609.pdf',
 '00194914_f_20201223125616.pdf',
 '00194915_f_20201212000711.pdf']

In [111]:
gp_name = []
district = []
total_pop = []
male_pop = []
female_pop = []
poverty_score = []
edu_score = []
health_score = []
total_score = []

exception_files = []
for file in files:
    pdf_file = "Antyodaya_Telangana/"+file
    tables = camelot.read_pdf(pdf_file, flavor='lattice', pages='1')
    df = tables[0].df
    try:
        gp_name.append(df.iloc[4][14][:-8])
        district.append(df.iloc[2][30].split('(')[0]) 
        total_pop.append(float(df.iloc[7][4]))
        male_pop.append(float(df.iloc[7][8]))
        female_pop.append(float(df.iloc[7][12]))
        poverty_score.append(float(df.iloc[13][37]))
        edu_score.append(float(df.iloc[13][25]))
        health_score.append(float(df.iloc[13][29]))
        total_score.append(float(df.iloc[13][45]))
    except:
        #print(file)
        exception_files.append(file)
        pass
    
## Then re-run the above loop with exception_files after making suitable changes. 

In [89]:
antyodaya_df = pd.DataFrame([gp_name,district,total_pop,male_pop,female_pop,poverty_score,edu_score,health_score,total_score]).T
antyodaya_df.columns = ['Gram_Panchayat','District','Total_Population','Male_Population','Female_Population','Poverty_Score','Education_Score','Health_Score','Total_Score']

In [117]:
antyodaya_df.dropna('').tail()

,Gram_Panchayat,District,Total_Population,Male_Population,Female_Population,Poverty_Score,Education_Score,Health_Score,Total_Score
8599,Pedda Thanda,JOGULAMBA GADWAL,616,318,298,3,2,9,48
8600,Gorloni Bavi,Mulugu,790,421,369,3,2,4.5,30.5
8601,Adluru(K),Mulugu,747,378,369,4,1,5,42
8602,Dhakya Thanda,YADADRI BHUVANAGIRI,568,306,262,2,1,5,34
8603,School Thanda,YADADRI BHUVANAGIRI,821,412,409,4,1,4.5,28


In [114]:
antyodaya_df.dropna().to_csv('TG_GP_Antyodaya.csv',index=False)